In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import input_data

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST ready")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
MNIST ready


In [3]:
n_input  = 784
n_output = 10
weights  = {
        'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.1)),
        'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.1)),
        'wd1': tf.Variable(tf.random_normal([7*7*128, 1024], stddev=0.1)),
        'wd2': tf.Variable(tf.random_normal([1024, n_output], stddev=0.1))
    }
biases   = {
        'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
        'bc2': tf.Variable(tf.random_normal([128], stddev=0.1)),
        'bd1': tf.Variable(tf.random_normal([1024], stddev=0.1)),
        'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1))
    }

In [4]:
def conv_basic(_input, _w, _b, _keepratio):
        # INPUT
        _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
        # CONV LAYER 1
        _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
        #_mean, _var = tf.nn.moments(_conv1, [0, 1, 2])
        #_conv1 = tf.nn.batch_normalization(_conv1, _mean, _var, 0, 1, 0.0001)
        _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1, _b['bc1']))
        _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
        # CONV LAYER 2
        _conv2 = tf.nn.conv2d(_pool_dr1, _w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
        #_mean, _var = tf.nn.moments(_conv2, [0, 1, 2])
        #_conv2 = tf.nn.batch_normalization(_conv2, _mean, _var, 0, 1, 0.0001)
        _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2, _b['bc2']))
        _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
        # VECTORIZE
        _dense1 = tf.reshape(_pool_dr2, [-1, _w['wd1'].get_shape().as_list()[0]])
        # FULLY CONNECTED LAYER 1
        _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
        _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
        # FULLY CONNECTED LAYER 2
        _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
        # RETURN
        out = { 'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1, 'pool1_dr1': _pool_dr1,
            'conv2': _conv2, 'pool2': _pool2, 'pool_dr2': _pool_dr2, 'dense1': _dense1,
            'fc1': _fc1, 'fc_dr1': _fc_dr1, 'out': _out
        }
        return out
print ("CNN READY")

CNN READY


In [11]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

# FUNCTIONS

_pred = conv_basic(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred,labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) 
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) 
init = tf.global_variables_initializer()
    
# SAVER
save_step = 1
saver = tf.train.Saver(max_to_keep=3) 

print ("GRAPH READY")

GRAPH READY


In [12]:
do_train = 1
sess = tf.Session()
sess.run(init)

In [ ]:
training_epochs = 1500
batch_size      = 128
display_step    = 1
if do_train == 1:
    for epoch in range(training_epochs):
        avg_cost = 0.
        #total_batch = int(mnist.train.num_examples/batch_size)
        total_batch = 10
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio:0.7})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})/total_batch

        # Display logs per epoch step
        if epoch % display_step == 0: 
            print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
            train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})
            print (" Training accuracy: %.3f" % (train_acc))
            #test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel, keepratio:1.})
            #print (" Test accuracy: %.3f" % (test_acc))
            
        # Save Net
        if epoch % save_step == 0:
            saver.save(sess, "save/nets/cnn_mnist_basic.ckpt-" + str(epoch))

    print ("OPTIMIZATION FINISHED")

Epoch: 000/1500 cost: 0.388099372
 Training accuracy: 0.938
Epoch: 001/1500 cost: 0.368950877
 Training accuracy: 0.891
Epoch: 002/1500 cost: 0.367267370
 Training accuracy: 0.891
Epoch: 003/1500 cost: 0.353424296
 Training accuracy: 0.961
Epoch: 004/1500 cost: 0.301800860
 Training accuracy: 0.961
Epoch: 005/1500 cost: 0.292834382
 Training accuracy: 0.914
Epoch: 006/1500 cost: 0.282511014
 Training accuracy: 0.938
Epoch: 007/1500 cost: 0.210005870
 Training accuracy: 0.938
Epoch: 008/1500 cost: 0.228892374
 Training accuracy: 0.938
Epoch: 009/1500 cost: 0.225452551
 Training accuracy: 0.906
Epoch: 010/1500 cost: 0.192881058
 Training accuracy: 0.922
Epoch: 011/1500 cost: 0.208348981
 Training accuracy: 0.922
Epoch: 012/1500 cost: 0.217334619
 Training accuracy: 0.930
Epoch: 013/1500 cost: 0.200076775
 Training accuracy: 0.984
Epoch: 014/1500 cost: 0.210286342
 Training accuracy: 0.938
Epoch: 015/1500 cost: 0.188270503
 Training accuracy: 0.977
Epoch: 016/1500 cost: 0.189714593
 Train

Epoch: 137/1500 cost: 0.029569004
 Training accuracy: 0.977
Epoch: 138/1500 cost: 0.055424894
 Training accuracy: 0.992
Epoch: 139/1500 cost: 0.033582172
 Training accuracy: 0.992
Epoch: 140/1500 cost: 0.044062912
 Training accuracy: 1.000
Epoch: 141/1500 cost: 0.031590667
 Training accuracy: 0.977
Epoch: 142/1500 cost: 0.031502198
 Training accuracy: 1.000
Epoch: 143/1500 cost: 0.027699036
 Training accuracy: 1.000
Epoch: 144/1500 cost: 0.022612520
 Training accuracy: 0.992
Epoch: 145/1500 cost: 0.021204154
 Training accuracy: 1.000
Epoch: 146/1500 cost: 0.032010093
 Training accuracy: 0.992
Epoch: 147/1500 cost: 0.029611997
 Training accuracy: 0.984
Epoch: 148/1500 cost: 0.027082864
 Training accuracy: 1.000
Epoch: 149/1500 cost: 0.023888612
 Training accuracy: 0.984
Epoch: 150/1500 cost: 0.042332623
 Training accuracy: 0.984
Epoch: 151/1500 cost: 0.027017449
 Training accuracy: 0.992
Epoch: 152/1500 cost: 0.019965532
 Training accuracy: 0.984
Epoch: 153/1500 cost: 0.029106825
 Train

Epoch: 274/1500 cost: 0.011450522
 Training accuracy: 1.000
Epoch: 275/1500 cost: 0.017333835
 Training accuracy: 0.984
Epoch: 276/1500 cost: 0.013101086
 Training accuracy: 1.000
Epoch: 277/1500 cost: 0.018496411
 Training accuracy: 1.000
Epoch: 278/1500 cost: 0.014416455
 Training accuracy: 1.000
Epoch: 279/1500 cost: 0.016095231
 Training accuracy: 0.992
Epoch: 280/1500 cost: 0.018802020
 Training accuracy: 1.000
Epoch: 281/1500 cost: 0.020700604
 Training accuracy: 0.992
Epoch: 282/1500 cost: 0.009519498
 Training accuracy: 0.992
Epoch: 283/1500 cost: 0.028114235
 Training accuracy: 0.977
Epoch: 284/1500 cost: 0.016503998
 Training accuracy: 0.992
Epoch: 285/1500 cost: 0.015525217
 Training accuracy: 0.992
Epoch: 286/1500 cost: 0.011681238
 Training accuracy: 1.000
Epoch: 287/1500 cost: 0.016791927
 Training accuracy: 1.000
Epoch: 288/1500 cost: 0.016447150
 Training accuracy: 1.000
Epoch: 289/1500 cost: 0.020850347
 Training accuracy: 1.000
Epoch: 290/1500 cost: 0.015754251
 Train

Epoch: 411/1500 cost: 0.007064981
 Training accuracy: 1.000
Epoch: 412/1500 cost: 0.007280589
 Training accuracy: 1.000
Epoch: 413/1500 cost: 0.011870010
 Training accuracy: 1.000
Epoch: 414/1500 cost: 0.005061848
 Training accuracy: 1.000
Epoch: 415/1500 cost: 0.003171189
 Training accuracy: 1.000
Epoch: 416/1500 cost: 0.008372079
 Training accuracy: 1.000
Epoch: 417/1500 cost: 0.014527262
 Training accuracy: 1.000
Epoch: 418/1500 cost: 0.011193437
 Training accuracy: 0.984
Epoch: 419/1500 cost: 0.017693924
 Training accuracy: 1.000
Epoch: 420/1500 cost: 0.010138190
 Training accuracy: 1.000
Epoch: 421/1500 cost: 0.009809654
 Training accuracy: 0.992
Epoch: 422/1500 cost: 0.010793611
 Training accuracy: 1.000
Epoch: 423/1500 cost: 0.007810690
 Training accuracy: 1.000
Epoch: 424/1500 cost: 0.004611550
 Training accuracy: 1.000
Epoch: 425/1500 cost: 0.013487561
 Training accuracy: 1.000
Epoch: 426/1500 cost: 0.008437885
 Training accuracy: 1.000
Epoch: 427/1500 cost: 0.010807197
 Train

Epoch: 548/1500 cost: 0.007518323
 Training accuracy: 1.000
Epoch: 549/1500 cost: 0.004737313
 Training accuracy: 1.000
Epoch: 550/1500 cost: 0.004543574
 Training accuracy: 1.000
Epoch: 551/1500 cost: 0.006398759
 Training accuracy: 1.000
Epoch: 552/1500 cost: 0.003121147
 Training accuracy: 1.000
Epoch: 553/1500 cost: 0.006174331
 Training accuracy: 1.000
Epoch: 554/1500 cost: 0.002924828
 Training accuracy: 1.000
Epoch: 555/1500 cost: 0.005311370
 Training accuracy: 1.000
Epoch: 556/1500 cost: 0.004369123
 Training accuracy: 1.000
Epoch: 557/1500 cost: 0.006071780
 Training accuracy: 1.000
Epoch: 558/1500 cost: 0.011191174
 Training accuracy: 1.000
Epoch: 559/1500 cost: 0.001923642
 Training accuracy: 1.000
Epoch: 560/1500 cost: 0.001598725
 Training accuracy: 1.000
Epoch: 561/1500 cost: 0.003602477
 Training accuracy: 1.000
Epoch: 562/1500 cost: 0.004101880
 Training accuracy: 1.000
Epoch: 563/1500 cost: 0.004515254
 Training accuracy: 1.000
Epoch: 564/1500 cost: 0.010553907
 Train

Epoch: 685/1500 cost: 0.002613818
 Training accuracy: 1.000
Epoch: 686/1500 cost: 0.003937862
 Training accuracy: 1.000
Epoch: 687/1500 cost: 0.005852142
 Training accuracy: 1.000
Epoch: 688/1500 cost: 0.001444419
 Training accuracy: 1.000
Epoch: 689/1500 cost: 0.002769244
 Training accuracy: 1.000
Epoch: 690/1500 cost: 0.001731632
 Training accuracy: 1.000
Epoch: 691/1500 cost: 0.002303883
 Training accuracy: 1.000
Epoch: 692/1500 cost: 0.001310342
 Training accuracy: 1.000
Epoch: 693/1500 cost: 0.006499774
 Training accuracy: 1.000
Epoch: 694/1500 cost: 0.003352307
 Training accuracy: 0.992
Epoch: 695/1500 cost: 0.008009841
 Training accuracy: 0.992
Epoch: 696/1500 cost: 0.003299204
 Training accuracy: 1.000
Epoch: 697/1500 cost: 0.001437482
 Training accuracy: 1.000
Epoch: 698/1500 cost: 0.006651336
 Training accuracy: 0.992
Epoch: 699/1500 cost: 0.003047735
 Training accuracy: 1.000
Epoch: 700/1500 cost: 0.004758208
 Training accuracy: 0.992
Epoch: 701/1500 cost: 0.001846195
 Train

Epoch: 822/1500 cost: 0.000472059
 Training accuracy: 1.000
Epoch: 823/1500 cost: 0.000763484
 Training accuracy: 1.000
Epoch: 824/1500 cost: 0.001281831
 Training accuracy: 1.000
Epoch: 825/1500 cost: 0.002391351
 Training accuracy: 0.992
Epoch: 826/1500 cost: 0.003966329
 Training accuracy: 1.000
Epoch: 827/1500 cost: 0.000771632
 Training accuracy: 1.000
Epoch: 828/1500 cost: 0.001987617
 Training accuracy: 1.000
Epoch: 829/1500 cost: 0.001122420
 Training accuracy: 1.000
Epoch: 830/1500 cost: 0.000745189
 Training accuracy: 1.000
Epoch: 831/1500 cost: 0.000624005
 Training accuracy: 1.000
Epoch: 832/1500 cost: 0.002652548
 Training accuracy: 1.000
Epoch: 833/1500 cost: 0.000980301
 Training accuracy: 1.000
Epoch: 834/1500 cost: 0.001577069
 Training accuracy: 1.000
Epoch: 835/1500 cost: 0.002319098
 Training accuracy: 1.000
Epoch: 836/1500 cost: 0.003628354
 Training accuracy: 1.000
Epoch: 837/1500 cost: 0.001074334
 Training accuracy: 1.000
Epoch: 838/1500 cost: 0.003659646
 Train

Epoch: 959/1500 cost: 0.001294022
 Training accuracy: 1.000
Epoch: 960/1500 cost: 0.000844360
 Training accuracy: 1.000
Epoch: 961/1500 cost: 0.001231116
 Training accuracy: 1.000
Epoch: 962/1500 cost: 0.002444434
 Training accuracy: 1.000
Epoch: 963/1500 cost: 0.001414868
 Training accuracy: 1.000
Epoch: 964/1500 cost: 0.001798474
 Training accuracy: 1.000
Epoch: 965/1500 cost: 0.001584536
 Training accuracy: 1.000
Epoch: 966/1500 cost: 0.004092522
 Training accuracy: 1.000
Epoch: 967/1500 cost: 0.001344809
 Training accuracy: 1.000
Epoch: 968/1500 cost: 0.001987024
 Training accuracy: 1.000
Epoch: 969/1500 cost: 0.007430110
 Training accuracy: 1.000
Epoch: 970/1500 cost: 0.001387538
 Training accuracy: 1.000
Epoch: 971/1500 cost: 0.000889481
 Training accuracy: 1.000
Epoch: 972/1500 cost: 0.000888724
 Training accuracy: 1.000
Epoch: 973/1500 cost: 0.001599306
 Training accuracy: 1.000
Epoch: 974/1500 cost: 0.002213122
 Training accuracy: 1.000
Epoch: 975/1500 cost: 0.000856476
 Train

Epoch: 1094/1500 cost: 0.001623893
 Training accuracy: 1.000
Epoch: 1095/1500 cost: 0.000417797
 Training accuracy: 1.000
Epoch: 1096/1500 cost: 0.000434701
 Training accuracy: 1.000
Epoch: 1097/1500 cost: 0.000741191
 Training accuracy: 1.000
Epoch: 1098/1500 cost: 0.000731753
 Training accuracy: 1.000
Epoch: 1099/1500 cost: 0.000600704
 Training accuracy: 1.000
Epoch: 1100/1500 cost: 0.004138800
 Training accuracy: 1.000
Epoch: 1101/1500 cost: 0.002475199
 Training accuracy: 1.000
Epoch: 1102/1500 cost: 0.002374441
 Training accuracy: 1.000
Epoch: 1103/1500 cost: 0.000641172
 Training accuracy: 1.000
Epoch: 1104/1500 cost: 0.000354109
 Training accuracy: 1.000
Epoch: 1105/1500 cost: 0.000226651
 Training accuracy: 1.000
Epoch: 1106/1500 cost: 0.000575598
 Training accuracy: 1.000
Epoch: 1107/1500 cost: 0.000599920
 Training accuracy: 1.000
Epoch: 1108/1500 cost: 0.000269148
 Training accuracy: 1.000
Epoch: 1109/1500 cost: 0.000964827
 Training accuracy: 1.000
Epoch: 1110/1500 cost: 0

Epoch: 1229/1500 cost: 0.000318572
 Training accuracy: 1.000
Epoch: 1230/1500 cost: 0.001908154
 Training accuracy: 1.000
Epoch: 1231/1500 cost: 0.000399839
 Training accuracy: 1.000
Epoch: 1232/1500 cost: 0.001120150
 Training accuracy: 1.000
Epoch: 1233/1500 cost: 0.000517577
 Training accuracy: 1.000
Epoch: 1234/1500 cost: 0.001835488
 Training accuracy: 1.000
Epoch: 1235/1500 cost: 0.000472696
 Training accuracy: 1.000
Epoch: 1236/1500 cost: 0.001280884
 Training accuracy: 1.000
Epoch: 1237/1500 cost: 0.000791050
 Training accuracy: 1.000
Epoch: 1238/1500 cost: 0.000352988
 Training accuracy: 1.000
Epoch: 1239/1500 cost: 0.000766400
 Training accuracy: 1.000
Epoch: 1240/1500 cost: 0.000882276
 Training accuracy: 1.000
Epoch: 1241/1500 cost: 0.001139490
 Training accuracy: 1.000
Epoch: 1242/1500 cost: 0.000307734
 Training accuracy: 1.000
Epoch: 1243/1500 cost: 0.000643897
 Training accuracy: 1.000
Epoch: 1244/1500 cost: 0.000252783
 Training accuracy: 1.000
Epoch: 1245/1500 cost: 0

In [ ]:
if do_train == 0:
    epoch = training_epochs-1
    saver.restore(sess, "save/nets/cnn_mnist_basic.ckpt-" + str(epoch))
    
    test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel, keepratio:1.})
    print (" TEST ACCURACY: %.3f" % (test_acc))